<a href="https://colab.research.google.com/github/amorfinv/icrat_notebooks/blob/main/street_data_scenario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using street data in BlueSky

The first step is to import all the necessary libraries and install any missing ones.

In [1]:
import os
import subprocess

try:
    import osmnx as ox
except ImportError:
    print('Installing osmnx ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'osmnx'])
    import osmnx as ox

try:
    import leafmap
except ImportError:
    print('Installing leafmap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'leafmap'])
    import leafmap

import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon, MultiLineString
from shapely import ops
import numpy as np

Here we create a map for our area of interest.

In [16]:
# Show map of Tampa
map_args = {"center":(27.95056, -82.45656), "zoom":16, "height":"500px", "width":"500px"}
cartomap = {
    "url":"https://cartodb-basemaps-b.global.ssl.fastly.net/light_nolabels/{z}/{x}/{y}.png",
    "name":"CartoDB",
    "attribution":"CartoDB",
}
m = leafmap.Map(**map_args)
m.add_tile_layer(**cartomap)
display(m)

Map(center=[27.95056, -82.45656], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

Select an origin and destination point and plot it

In [17]:
# Create a mission from pharmacy to firefighter
origin_point = Point(-82.45863, 27.94853)
destination_point = Point(-82.45413, 27.95102)

# dictionary for geodataframe
data = {'type': ['origin', 'destination'], 
        'geometry': [origin_point, destination_point],
        'latitude': [27.94853, 27.95102],
        'longitude': [-82.45863, -82.45413]
        }
gdf_od = gpd.GeoDataFrame(data, crs='epsg:4326')

# create map and add gdf
m = leafmap.Map(**map_args)
m.add_tile_layer(**cartomap)
m.add_points_from_xy(gdf_od, popup='type', layer_name='od', zoom_to_layer=False)
display(m)

Map(center=[27.95056, -82.45656], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

Create a graph with a polygon

In [19]:
# download OSM as a networkx graph

# create a polygon of area of interest
p1 = Point(-82.46079, 27.95042)
p2 = Point(-82.45905, 27.94679)
p3 = Point(-82.45168, 27.94934)
p4 = Point(-82.45344, 27.95290)

polygon = Polygon([p1, p2, p3, p4, p1])

# download graph of area
G = ox.graph_from_polygon(polygon, network_type='drive')

# Convert to gdf
nodes, edges = ox.graph_to_gdfs(G)
nodes['osmid'] = nodes.index
nodes.reset_index(drop=True, inplace=True)

# Show map
map_args = {"center":(27.95056, -82.45656), "zoom":17, "height":"800px", "width":"1000px"}
m = leafmap.Map(**map_args)
m.add_tile_layer(**cartomap)
m.add_points_from_xy(gdf_od, popup='type', layer_name='od', zoom_to_layer=False)
m.add_points_from_xy(nodes, x="x", y="y", layer_name='nodes', zoom_to_layer=False, popup='osmid')
m.add_gdf(edges, layer_name='edges', style={"color": "black", "weight": 3}, zoom_to_layer=False)

display(m)

Map(center=[27.95056, -82.45656], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

Get the route with an origin and desitination points

In [22]:
# get a route from origin to destination
X = [origin_point.x, destination_point.x]
Y = [origin_point.y, destination_point.y]

# find the nearest nodes to the origin and destination points
orig, dest = ox.nearest_nodes(G, X, Y, return_dist=False)

# Get the route as a list of nodes
route = ox.shortest_path(G, orig, dest)

# get gemetry of route and put in a gdf
route_geom = ox.utils_graph.get_route_edge_attributes(G, route, attribute="geometry")
multi_line = MultiLineString(route_geom)
route_gdf = gpd.GeoDataFrame(geometry=[ops.linemerge(multi_line)], crs="epsg:4326")

# Show map
m = leafmap.Map(**map_args)
m.add_tile_layer(**cartomap)
m.add_points_from_xy(gdf_od, x="longitude", y="latitude", popup='type', layer_name='od', zoom_to_layer=False)
m.add_gdf(edges, layer_name='edges', style={"color": "black", "weight": 3}, zoom_to_layer=False)
m.add_gdf(route_gdf, layer_name='route', style={"color": "red", "weight": 6}, zoom_to_layer=False)
display(m)


Map(center=[27.95056, -82.45656], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

Now with the route and graph we can create a BlueSky scenario

In [6]:
# now we can create a bluesky scenario with this route
import tools

lats, lons, *_ = tools.get_lat_lon_from_osm_route(G, route)
turn_bool, *_ = tools.get_turn_arrays(lats, lons)

# create the scenario
scenario = tools.create_scenario_text(lats, lons, turn_bool)

# save the scenario
with open('D1.scn', "w") as f:
  f.write(scenario)

print(scenario)

# Allow BlueSky to understand 0 speed commands
00:00:00>CASMACHTHR 0.0

# Zoom and turn on tiledmap
00:00:00>ZOOM 350
00:00:00>VIS MAP TILEDMAP

# Create drone at height 0 speed 0 and pan to it
00:00:00>CRE D1 M600 27.9484372 -82.4582366 68.70043011899615 0 0
00:00:00>PAN D1

# Tell drone to climb to 60 feet
00:00:00>ALT D1 60

# At 60 feet drone should increase speed to 30 knots
00:00:00>D1 ATALT 60 SPD D1 30

# Add waypoints
00:00:00>ADDWAYPOINTS D1,27.9484652 -82.4581553,,30,FLYBY, 0,27.948628 -82.457683,,30,FLYBY, 0,27.9486836 -82.4575218,,30,FLYBY, 0,27.9487174 -82.4574238,,30,TURNSPD,10,27.9488104 -82.4574634,,30,FLYBY, 0,27.9490581 -82.4575684,,30,FLYBY, 0,27.949389 -82.4577099,,30,FLYBY, 0,27.949467 -82.457742,,30,TURNSPD,10,27.9495045 -82.457635,,30,FLYBY, 0,27.9497307 -82.4569898,,30,FLYBY, 0,27.949759 -82.456904,,30,FLYBY, 0,27.9497928 -82.4568055,,30,FLYBY, 0,27.949959 -82.45632,,30,FLYBY, 0,27.9500109 -82.4561667,,30,FLYBY, 0,27.950046 -82.456066,,30,FLYBY, 0,27.9500819 -8